#Improving SAT Participation
DSI CC7 - Project 1
Notebook 1 - Data Importing and Cleaning<br>
Anne Kerr - SF

### Project Introduction
This was the first project for the General Assembly Data Science Immersion Program. The assignement was to analyze the provided SAT and ACT datasets, augment the analysis with outside researchm, and create a recommendation regarding how to increase participation rates in future years.

### Problem Statement
Given changes in the SAT test itself, as well as the standardized testing landscape in general, how might College Board work to increase the participation rate?


### Overview of the Data 
Test Participation Rates and Scores
-  2017 SAT and ACT Datasets provided by College Board
-  2018 SAT and ACT Datasets prepared by General Assembly Data Immersion Students from 
-  Collected Classroom Cohort 6
Standardized Test Landscape
-  Trends for various state-wide requirements
-  Supplementary data publicly available - See references



### This Project Contains the Following Notebooks
- 1. Importing and Cleaning 
This notebook reads both the SAT and ACT data and performs a variety of functions to examine, and where necessary, clean the data.
- 2. Analysis and Conclusion
This notebook performs the analysis, summarizes the outside research, and outlines specific recommendations to College Board based on this study.

### Summary of Conclusion

in order to increase participation rates, I recommend that College Board to the following:

Follow Trends: Watch how the states are responding to Common Core requirements
Build and Strengthen State Relationships - Help them meet succeed 
Consider offering a Common Core test











In [1]:
#load the data science libraries

import pandas as pd
%matplotlib inline
import seaborn as sns
import numpy as np
import scipy as sp
import math

import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [2]:
###Read the datafiles in pandas dataframs
dfsat = pd.read_csv('../data/sat_2017.csv')
dfact = pd.read_csv('../data/act_2017.csv')

Now that the data is loaded, let's take a look at it....

Print the first 10 rows of each dataframe 

In [3]:
dfsat.head(10)

,State,Participation,Evidence-Based Reading and Writing,Math,Total
0,Alabama,5%,593,572,1165
1,Alaska,38%,547,533,1080
2,Arizona,30%,563,553,1116
3,Arkansas,3%,614,594,1208
4,California,53%,531,524,1055
5,Colorado,11%,606,595,1201
6,Connecticut,100%,530,512,1041
7,Delaware,100%,503,492,996
8,District of Columbia,100%,482,468,950
9,Florida,83%,520,497,1017


In [4]:
dfact.head(10)

,State,Participation,English,Math,Reading,Science,Composite
0,National,60%,20.3,20.7,21.4,21.0,21.0
1,Alabama,100%,18.9,18.4,19.7,19.4,19.2
2,Alaska,65%,18.7,19.8,20.4,19.9,19.8
3,Arizona,62%,18.6,19.8,20.1,19.8,19.7
4,Arkansas,100%,18.9,19.0,19.7,19.5,19.4
5,California,31%,22.5,22.7,23.1,22.2,22.8
6,Colorado,100%,20.1,20.3,21.2,20.9,20.8
7,Connecticut,31%,25.5,24.6,25.6,24.6,25.2
8,Delaware,18%,24.1,23.4,24.8,23.6,24.1
9,District of Columbia,32%,24.4,23.5,24.9,23.5,24.2


Use .shape and .info to get a sense of the overall dataset

In [5]:
print('SAT data shape is {}'.format(dfsat.shape))
print('ACT data shape is {}'.format(dfact.shape))

SAT data shape is (51, 5)
ACT data shape is (52, 7)


In [6]:
dfsat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 5 columns):
State                                 51 non-null object
Participation                         51 non-null object
Evidence-Based Reading and Writing    51 non-null int64
Math                                  51 non-null int64
Total                                 51 non-null int64
dtypes: int64(3), object(2)
memory usage: 2.1+ KB


In [7]:
dfact.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 7 columns):
State            52 non-null object
Participation    52 non-null object
English          52 non-null float64
Math             52 non-null float64
Reading          52 non-null float64
Science          52 non-null float64
Composite        52 non-null object
dtypes: float64(4), object(3)
memory usage: 2.9+ KB


The ACT has an additional row. Visual inspection shows that it has a row for National averages that SAT does not. I don't need it for the project, so I will remove it during cleanup. This will leave 51 rows, one for each state, plus the District of Columbia. There are no missing elements.

Let's explore the data a few different ways...

-  Checking the types of both files using .dtypes we see some columns listed as object types. This means there are some string values that need to be cleaned.


In [8]:
dfsat.dtypes

State                                 object
Participation                         object
Evidence-Based Reading and Writing     int64
Math                                   int64
Total                                  int64
dtype: object

In [9]:
dfact.dtypes

State             object
Participation     object
English          float64
Math             float64
Reading          float64
Science          float64
Composite         object
dtype: object

The following function was written to check that the ACT and SAT scores are in the range of valid scores or each test respectively. I wrote in generically so it could be used in future projects to check the range of other variables.

In [10]:
#define function to check if a value is in a range, inclusive
#inputs n: a nunber to check, min value 
def check_range(n,nmin,nmax):
    if n >= nmin and n <= nmax:
        return True
    else:
        print('Value out of range: {}'.format(n))
        return False
    

Use the check range function to validate the total and individual test scores in the file. If any are out of range, this function will find them.

In [11]:
### Valid SAT scores range from 200 to 800
## Check that the min is not below 200 and the max is not above 800    
print('SAT Math min value in range: {}'.format(check_range(dfsat['Math'].min(),200,800)))
print('SAT Math max value in range: {}'.format(check_range(dfsat['Math'].max(),200,800)))
print('SAT Math min value in range: {}'.format(check_range(dfsat['Evidence-Based Reading and Writing'].min(),200,800)))
print('SAT Math max value in range: {}'.format(check_range(dfsat['Evidence-Based Reading and Writing'].max(),200,800)))
### Valid ACT scores range from 1 - 36

## Check that the min is not below 1 and the max is not above 36
print('ACT Math min value in range: {}'.format(check_range(dfact['Math'].min(),1,36)))
print('ACT Math max value in range: {}'.format(check_range(dfact['Math'].max(),1,36)))

print('ACT English min value in range: {}'.format(check_range(dfact['English'].min(),1,36)))
print('ACT English max value in range: {}'.format(check_range(dfact['English'].max(),1,36)))

print('ACT Reading min value in range: {}'.format(check_range(dfact['Reading'].min(),1,36)))
print('ACT Reading max value in range: {}'.format(check_range(dfact['Reading'].max(),1,36)))

print('ACT Science min value in range: {}'.format(check_range(dfact['Science'].min(),1,36)))
print('ACT Science max value in range: {}'.format(check_range(dfact['Science'].max(),1,36)))

Value out of range: 52
SAT Math min value in range: False
SAT Math max value in range: True
SAT Math min value in range: True
SAT Math max value in range: True
ACT Math min value in range: True
ACT Math max value in range: True
ACT English min value in range: True
ACT English max value in range: True
ACT Reading min value in range: True
ACT Reading max value in range: True
ACT Science min value in range: True
ACT Science max value in range: True


The above reveals a problem with the SAT Math data. One entry is out of range. By referencing the SAT web site I realized the entry should be 524. I will modify it as part of data cleaning.

### Data Cleaning
Tasks include:
-  Fixing problems in numeric columns (Participation in both files, composite in SAT)   
-  Converting those columns to the correct numeric type 
-  Fixing the typo in the SAT math column
    

In [12]:
##Fix the SAT math problem
##Mark helped me with this
dfsat[dfsat['Math'] == 52]

,State,Participation,Evidence-Based Reading and Writing,Math,Total
20,Maryland,69%,536,52,1060


In [13]:
idx = dfsat[dfsat['Math'] == 52].index.values #get the index of the row with 'Math' == 52
idx

array([20])

In [14]:
dfsat.loc[idx,'Math'] 

20    52
Name: Math, dtype: int64

In [15]:
dfsat.loc[idx,'Math'] = 524

In [16]:
dfsat.loc[idx,'Math']   #check that we have changed the value

20    524
Name: Math, dtype: int64

The math column is fixed. Now fix the participatimon columns. In both files the format of this column includes the percent sign. I need to strip these off before I change them to numeric.

In [17]:
dfsat['Participation'] = dfsat['Participation'].str.replace('%','')
dfsat['Participation'] = pd.to_numeric(dfsat['Participation'])
dfsat.dtypes


State                                 object
Participation                          int64
Evidence-Based Reading and Writing     int64
Math                                   int64
Total                                  int64
dtype: object

In [18]:
dfact['Participation'] = dfact['Participation'].str.replace('%','')
dfact['Participation'] = pd.to_numeric(dfact['Participation'])
dfact.dtypes


State             object
Participation      int64
English          float64
Math             float64
Reading          float64
Science          float64
Composite         object
dtype: object

Inspect the composite column to print any non-numeric values:

In [19]:
for n in dfact['Composite']:
    try:
        x = float(n)
    except:
        print(n)


20.2x


In [20]:
#get the index of any value that is not numeric
idx = dfact[dfact['Composite'] == '20.2x'].index.values
#Replace it 
dfact.loc[idx,'Composite'] = '20.2'
#Now we can convert to numeric
dfact['Composite'] = pd.to_numeric(dfact['Composite'])
#Display the data types again to confirm they are correct.
dfact.dtypes

State             object
Participation      int64
English          float64
Math             float64
Reading          float64
Science          float64
Composite        float64
dtype: object

In [21]:
#The ACT file has a national row
#We will eliminate this from our dataset

dfact = dfact.drop(dfact.loc[dfact['State'] == 'National'].index.values)
len(dfact)


51

## Rename Columns
We have been asked to change the names of the columns to more expressive names so that we can tell the difference between the SAT columns and the ACT columns. We plan to combine the files, and have been given the following guidelines:



-  Column names should be all lowercase 
-  Column names should not contain spaces (underscores will suffice--this allows for using the df.column_name method to access columns in addition to df['column_name'].
-  Column names should be unique and informative 

In [22]:
#Rename SAT columns
dfsat = dfsat.rename(columns = {'State':'state',
                                'Participation':'sat_participation_2017',
                                 'Evidence-Based Reading and Writing':'sat_language_2017',
                                 'Math':'sat_math_2017',
                                 'Total':'sat_total_2017'
                                })

dfsat.columns

Index(['state', 'sat_participation_2017', 'sat_language_2017', 'sat_math_2017',
       'sat_total_2017'],
      dtype='object')

In [23]:
#Rename ACT columns
dfact = dfact.rename(columns = {'State':'state',
                                 'Participation':'act_participation_2017',
                                 'English':'act_english_2017',
                                 'Math':'act_math_2017',
                                 'Reading':'act_reading_2017',
                                 'Science':'act_science_2017',
                                 'Composite':'act_composite_2017'
                                })

dfact.columns

Index(['state', 'act_participation_2017', 'act_english_2017', 'act_math_2017',
       'act_reading_2017', 'act_science_2017', 'act_composite_2017'],
      dtype='object')

## Data Dictionary

The following table describes the features of the combined SAT/ACT dataset

|Feature|Type|Dataset|Description|
|---|---|---|---| 
|state|Object|Both|State Name|
|sat_participation_2017|int64|SAT|Percent class of 2017 who took the SAT|
|sat_language_2017|int64|SAT|Average SAT language score 2017|
|sat_math_2017|int64|SAT|Average SAT math score 2017|
|sat_total_2017|int64|SAT|Average SAT total score 2017|
|act_participation_2017|int64|ACT|Percent class of 2017 who took the ACT|
|act_english_2017|float64|ACT|Average ACT english score 2017|
|act_math_2017|float64|ACT|Average ACT math score 2017|
|act_reading_2017|float64|ACT|Average ACT reading score 2017|
|act_science_2017|float64|ACT|Average ACT scienc score 2017|
|act_composite_2017|float64|ACT|Average ACT Composite score 2017|


I am now ready to....
## Combine the datasets

In [24]:
#Merge on state
#thank you stack overflow: https://stackoverflow.com/
#questions/30584486/join-two-pandas-dataframe-using-a-specific-column
df = dfsat.merge(dfact, on='state', how='inner')
df.head()

,state,sat_participation_2017,sat_language_2017,sat_math_2017,sat_total_2017,act_participation_2017,act_english_2017,act_math_2017,act_reading_2017,act_science_2017,act_composite_2017
0,Alabama,5,593,572,1165,100,18.9,18.4,19.7,19.4,19.2
1,Alaska,38,547,533,1080,65,18.7,19.8,20.4,19.9,19.8
2,Arizona,30,563,553,1116,62,18.6,19.8,20.1,19.8,19.7
3,Arkansas,3,614,594,1208,100,18.9,19.0,19.7,19.5,19.4
4,California,53,531,524,1055,31,22.5,22.7,23.1,22.2,22.8


In [25]:
df.columns

Index(['state', 'sat_participation_2017', 'sat_language_2017', 'sat_math_2017',
       'sat_total_2017', 'act_participation_2017', 'act_english_2017',
       'act_math_2017', 'act_reading_2017', 'act_science_2017',
       'act_composite_2017'],
      dtype='object')

## Save the data
Finally, save the 2017 dataset

In [26]:
#Save the 2017 data
df.to_csv('../data/sat_act_2017.csv')

I now need to get 2018 data, clean it, and merge it in. Thanks to DSI CC6 we have a spreadsheet to use as a starting point. The spreadhseets contains a tab each for the SAT and ACT data, so I saved them to separate csv files for importing into our project. I will load them now.

In [27]:
#Read the 2018 SAT Data, courtesy of DSI cc6
dfsat2018 = pd.read_csv('../data/2018 SAT Data.csv')
dfsat2018.head()

 #Read the 2018 ACT Data, courtesy of DSI cc6
dfact2018 = pd.read_csv('../data/2018 ACT Data.csv')
dfact2018.head()

,State,Participation,Composite
0,Alabama,100%,19.1
1,Alaska,33%,20.8
2,Arizona,66%,19.2
3,Arkansas,100%,19.4
4,California,27%,22.7


It seems we only have composite data for 2018 ACT. That should be okay for our project, though.

I wll perform similar exploration and data cleaning as was done for the 2017 data.

I now know we need to rename the columns, so I'll do that first.

In [28]:
#Rename the columns of both 2018 datasets
dfsat2018 = dfsat2018.rename(columns = {'State':'state',
                                'Participation':'sat_participation_2018',
                                 'Evidence-Based Reading and Writing':'sat_language_2018',
                                 'Math':'sat_math_2018',
                                 'Total':'sat_total_2018'
                                })

print(dfsat2018.columns)


#Rename ACT columns
dfact2018 = dfact2018.rename(columns = {'State':'state',
                                 'Participation':'act_participation_2018',
                                 'Composite':'act_composite_2018'
                                })

dfact2018.columns

Index(['state', 'sat_participation_2018', 'sat_language_2018', 'sat_math_2018',
       'sat_total_2018'],
      dtype='object')


Index(['state', 'act_participation_2018', 'act_composite_2018'], dtype='object')

Are any scores out of range?

In [29]:
print('SAT Math min value in range: {}'.format(check_range(dfsat2018['sat_math_2018'].min(),200,800)))
print('SAT Math max value in range: {}'.format(check_range(dfsat2018['sat_math_2018'].max(),200,800)))
print('SAT Math min value in range: {}'.format(check_range(dfsat2018['sat_language_2018'].min(),200,800)))
print('SAT Math max value in range: {}'.format(check_range(dfsat2018['sat_language_2018'].max(),200,800)))


print('ACT Composite min value in range: {}'.format(check_range(dfact2018['act_composite_2018'].min(),1,36)))
print('ACT Composite max value in range: {}'.format(check_range(dfact2018['act_composite_2018'].max(),1,36)))


SAT Math min value in range: True
SAT Math max value in range: True
SAT Math min value in range: True
SAT Math max value in range: True
ACT Composite min value in range: True
ACT Composite max value in range: True


All good. What else do we see?  

In [30]:
#Check the types of the columns
print('\n2018 SAT')
print(dfsat2018.dtypes)
print('\n2018 ACT')
print(dfact2018.dtypes)


2018 SAT
state                     object
sat_participation_2018    object
sat_language_2018          int64
sat_math_2018              int64
sat_total_2018             int64
dtype: object

2018 ACT
state                      object
act_participation_2018     object
act_composite_2018        float64
dtype: object


I will have to change the participation columns again.

In [31]:
#strip the % signs
dfsat2018['sat_participation_2018'] = dfsat2018['sat_participation_2018'].str.replace('%','')
dfact2018['act_participation_2018'] = dfact2018['act_participation_2018'].str.replace('%','')

#and convert the columns to numeric
dfsat2018['sat_participation_2018'] = pd.to_numeric(dfsat2018['sat_participation_2018'])
dfact2018['act_participation_2018'] = pd.to_numeric(dfact2018['act_participation_2018'])


There is a discrepancy in state names for DC. The 2018 file lists it as Washingon, D.C., wheras it is District of Columbia everywhere else. I also see that Oklahoma is misspelled in the SAT file. I will fix these errors next.

In [32]:
##Te 2018 ACT data lists Washington, D.C.. Change it to District of Columbia to be consistent
idx = dfact2018[dfact2018.state == 'Washington, D.C.'].index.values #get the index of the row to change
idx

print(dfact2018.loc[idx,'state'])   #check that we are pointing to the right one
dfact2018.loc[idx,'state'] = 'District of Columbia'
print(dfact2018.loc[idx,'state'])  #check that we have changed the value


47    Washington, D.C.
Name: state, dtype: object
47    District of Columbia
Name: state, dtype: object


In [33]:
#Fix spelling of Oklahoma in the SAT file
idx = dfsat2018[dfsat2018['state'] == 'Oaklahoma'].index.values 
dfsat2018.loc[idx,'state'] = 'Oklahoma' 


In [34]:
dfsat2018.loc[idx,'state']

36    Oklahoma
Name: state, dtype: object

Before merging, let's check the shape and info to see they are of the same dimension.

In [35]:
print('SAT data shape is {}'.format(dfsat2018.shape))
print('ACT data shape is {}'.format(dfact2018.shape))

SAT data shape is (51, 5)
ACT data shape is (52, 3)


In [36]:
dfsat2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 5 columns):
state                     51 non-null object
sat_participation_2018    51 non-null int64
sat_language_2018         51 non-null int64
sat_math_2018             51 non-null int64
sat_total_2018            51 non-null int64
dtypes: int64(4), object(1)
memory usage: 2.1+ KB


In [37]:
dfact2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 3 columns):
state                     52 non-null object
act_participation_2018    52 non-null int64
act_composite_2018        52 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.3+ KB


They are not. The 2018 ACT file also has a National row to be removed.

In [38]:
#Eliminate the national row from the ACT dataset
#dfact2018.loc[dfact2018['state'] == 'National'].index.values

dfact2018 = dfact2018.drop(dfact2018.loc[dfact2018['state'] == 'National'].index.values)
#it was at the end of the file, so using tail verifies it is now gone
dfact.tail()

,state,act_participation_2017,act_english_2017,act_math_2017,act_reading_2017,act_science_2017,act_composite_2017
47,Virginia,29,23.5,23.3,24.6,23.5,23.8
48,Washington,29,20.9,21.9,22.1,22.0,21.9
49,West Virginia,69,20.0,19.4,21.2,20.5,20.4
50,Wisconsin,100,19.7,20.4,20.6,20.9,20.5
51,Wyoming,100,19.4,19.8,20.8,20.6,20.2


## Combining 2017 and 2018 Data
We will now combine the data and save it as final.csv

In [39]:
#Merge in 2018 SAT Dat

df = df.merge(dfsat2018, on='state', how='inner')
df = df.merge(dfact2018, on='state', how='inner')
df.head()

,state,sat_participation_2017,sat_language_2017,sat_math_2017,sat_total_2017,act_participation_2017,act_english_2017,act_math_2017,act_reading_2017,act_science_2017,act_composite_2017,sat_participation_2018,sat_language_2018,sat_math_2018,sat_total_2018,act_participation_2018,act_composite_2018
0,Alabama,5,593,572,1165,100,18.9,18.4,19.7,19.4,19.2,6,595,571,1166,100,19.1
1,Alaska,38,547,533,1080,65,18.7,19.8,20.4,19.9,19.8,43,562,544,1106,33,20.8
2,Arizona,30,563,553,1116,62,18.6,19.8,20.1,19.8,19.7,29,577,572,1149,66,19.2
3,Arkansas,3,614,594,1208,100,18.9,19.0,19.7,19.5,19.4,5,592,576,1169,100,19.4
4,California,53,531,524,1055,31,22.5,22.7,23.1,22.2,22.8,60,540,536,1076,27,22.7


In [40]:
df.shape

(51, 17)

In [41]:
df.columns

Index(['state', 'sat_participation_2017', 'sat_language_2017', 'sat_math_2017',
       'sat_total_2017', 'act_participation_2017', 'act_english_2017',
       'act_math_2017', 'act_reading_2017', 'act_science_2017',
       'act_composite_2017', 'sat_participation_2018', 'sat_language_2018',
       'sat_math_2018', 'sat_total_2018', 'act_participation_2018',
       'act_composite_2018'],
      dtype='object')

In [42]:
df.head()

,state,sat_participation_2017,sat_language_2017,sat_math_2017,sat_total_2017,act_participation_2017,act_english_2017,act_math_2017,act_reading_2017,act_science_2017,act_composite_2017,sat_participation_2018,sat_language_2018,sat_math_2018,sat_total_2018,act_participation_2018,act_composite_2018
0,Alabama,5,593,572,1165,100,18.9,18.4,19.7,19.4,19.2,6,595,571,1166,100,19.1
1,Alaska,38,547,533,1080,65,18.7,19.8,20.4,19.9,19.8,43,562,544,1106,33,20.8
2,Arizona,30,563,553,1116,62,18.6,19.8,20.1,19.8,19.7,29,577,572,1149,66,19.2
3,Arkansas,3,614,594,1208,100,18.9,19.0,19.7,19.5,19.4,5,592,576,1169,100,19.4
4,California,53,531,524,1055,31,22.5,22.7,23.1,22.2,22.8,60,540,536,1076,27,22.7


In [43]:
df.tail()

,state,sat_participation_2017,sat_language_2017,sat_math_2017,sat_total_2017,act_participation_2017,act_english_2017,act_math_2017,act_reading_2017,act_science_2017,act_composite_2017,sat_participation_2018,sat_language_2018,sat_math_2018,sat_total_2018,act_participation_2018,act_composite_2018
46,Virginia,65,561,541,1102,29,23.5,23.3,24.6,23.5,23.8,68,567,550,1117,24,23.9
47,Washington,64,541,534,1075,29,20.9,21.9,22.1,22.0,21.9,69,543,538,1081,24,22.2
48,West Virginia,14,558,528,1086,69,20.0,19.4,21.2,20.5,20.4,28,513,486,999,65,20.3
49,Wisconsin,3,642,649,1291,100,19.7,20.4,20.6,20.9,20.5,3,641,653,1294,100,20.5
50,Wyoming,3,626,604,1230,100,19.4,19.8,20.8,20.6,20.2,3,633,625,1257,100,20.0


In [44]:
#Save the final combined dataset
df.to_csv('../data/final.csv', index=False)
#note, added index=False to prevent writing the index as data
#a general internet search turned up several suggestions to add index_col = False
#which didn't work, but the to_csv documentation provided the answer

### Data Cleaning Summary
The final datasets have been saved. I will perform the analysis. Please see the notebook named Analysis.ipynb.

